In [119]:
from numpy import *
from scipy.optimize import fsolve
from scipy.integrate import odeint
from scipy.stats import linregress 
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline

# <ins>Question 1 [14] </ins>

Consider the following reversible elementary liquid phase reaction:
    $$A \rightleftharpoons B$$
Rate and thermodynamic data of the reaction is given in the table below

<img src="CRO410Supp2020a.JPG" width="350" />

Pure A at $C_{A_{0}} = 100~mol/m^{3}$ with a volumetric flow rate of $ Q = 10 \times 10^{-3}~m{3}/s$ is converted in a reactor set-up consisting of two adiabatic CSTR’s in series. The same type of solid catalyst is used in each reactor. The inlet temperature to the first reactor is $T_{0} = 310~K$ and the conversion of A achieved in this reactor is $x_{A} = 40~\%$.

After the first reactor, the formed product $B$ is separated from the product stream, so that the resultant feed stream to the second reactor contains only pure $A$ again. The second reactor must be operated in such a way that an *overall conversion* of A of $85~\%$ is achieved. After separation, the inlet stream to $Reactor 2$ can be cooled or heated in a heat exchanger before processing in $Reactor 2$.

**Assumptions:**

- Densities of all liquids are equal and $\rho_{l} = 950~kg/m^{3}$
- All mass transfer effects are negligible
- The separation process does not result in a change in temperature

In [120]:
R     =  8.314
Kcref =   2000 #at 300K
dHrx  = -30000 #J/mol
CpA   =     80 # J/mol
CpB   =    CpA
kopr  =    193 #m3/kg/s
Ea    =  60000 #J/mol
rholiq=    950 #kg/m3 
n     =      1
CAo   =    100 #mol/m3
Q     =  10e-3 #m3/s
To    =    310 #K

*a) What is the concentration of component $A$ in the second reactor?  give in units of $mol/m^{3}$. [2]*

\begin{align}
(1 - X) &= (1 - X_{1})(1 - X_{2})\nonumber \\
\dfrac{(1 - X)}{(1 - X_{1})} &= (1 - X_{2}) \nonumber \\
X_{2} &= 1 - \dfrac{(1 - X)}{(1 - X_{1})} \nonumber
\end{align}

In [121]:
Xt = 0.85
X1 = 0.40
X2 = 1 - (1 - Xt)/(1- X1)
X2

0.75

In [122]:
CAin = CAo*(1 - X1)
CAin

60.0

*b) What is the concentration of component B in the second reactor? give in units of $mol/m3$ [2]*

In [123]:
CA = CAin*(1 - X2)
CB = CAin * X2
print(CA)
print(CB)

15.0
45.0


*c) Specify the amount of inter-cooling (give in $kW$, round to 3 significant numbers) necessary to minimize the mass of catalyst required in the second reactor to achieve the required conversion. [10]*

CSTR design equation:
\begin{align}
F_{A_{0}} - F_{A} + r_{A}^{'}W &= 0 \nonumber \\
F_{A_{0}}X_{A} &= -r_{A}^{'}W \nonumber \\
W &= \dfrac{F_{A_{0}}X_{A}}{-r_{A}^{'}} \nonumber
\end{align}

Rate equation:
\begin{align}
r_{A} &= r_{A}^{'}\rho_{l} \nonumber \\
r_{A}^{'}\rho_{l} &= -k_{1}\left( C_{A} - \dfrac{C_{B}}{K_{c}} \right) \nonumber \\
\dfrac{-F_{A_{0}}X\rho_{l}}{W} &= -k_{1}\left( C_{A} - \dfrac{C_{B}}{K_{c}} \right)  \nonumber \\
\dfrac{F_{A_{0}}X\rho_{l}}{k_{1}} &= W\left( C_{A} - \dfrac{C_{B}}{K_{c}} \right)  \nonumber \\
W &= \dfrac{F_{A_{0}}X\rho_{l}}{k_{1}}\left( C_{A} - \dfrac{C_{B}}{K_{c}} \right) ^{-1} \nonumber
\end{align}

In [124]:
T1 = To + (-dHrx/CpA)*X1
T1

460.0

From the rate at equilibrium $= 0$ (-r_{A} = 0):
\begin{align}
X_{eq} &= \dfrac{K_{eq}}{1 + K_{eq}} \nonumber \\
X_{eq}(1 + K_{eq}) &= K_{eq} \nonumber \\
X_{eq} &= K_{eq}(1 - X_{eq}) \nonumber \\
K_{eq} &= \dfrac{X_{eq}}{1 - X_{eq}} \nonumber
\end{align}

In [125]:
Kc = X2/(1 - X2)
Kc

3.0

From Van't Hoff's equation:
\begin{align}
K_{eq} &= K_{eq,ref}e^{\left[ \dfrac{-\Delta H_{A}}{R}\left(\dfrac{1}{T} - \dfrac{1}{T_{ref}} \right) \right]}\nonumber \\
ln\left(\dfrac{K_{eq}}{K_{eq,ref}}\right) &= {\left[ \dfrac{-\Delta H_{A}}{R}\left(\dfrac{1}{T} - \dfrac{1}{T_{ref}} \right) \right]}\nonumber \\
\left(\dfrac{1}{T} - \dfrac{1}{T_{ref}} \right) &= \dfrac{R}{-\Delta H_{A}}ln\left(\dfrac{K_{eq}}{K_{eq,ref}}\right) \nonumber \\
\dfrac{1}{T} &= \dfrac{1}{T_{ref}} -  \dfrac{R}{\Delta H_{A}}ln\left(\dfrac{K_{eq}}{K_{eq,ref}}\right) \nonumber \\
T &= \left[ \dfrac{1}{T_{ref}} -  \dfrac{R}{\Delta H_{A}}ln\left(\dfrac{K_{eq}}{K_{eq,ref}}\right)\right]^{-1} \nonumber
\end{align}

In [126]:
a = 1/300
b = R/dHrx
c = log(Kc/Kcref)
Tout = (a - b*c)**(-1)
Tout

653.0262610998047

In [127]:
Tin = Tout + (dHrx/CpA)*X2
Tin

371.77626109980474

In [128]:
kiref   =  kopr*exp(-Ea/(R*To))
ki      =  kiref*exp((-Ea/R)*((1/Tout)-(1/To)))
Kc      =  Kcref*exp((-dHrx/R)*((1/Tout)-(1/300)))
r       =  ki*(CA - CB/Kc)
FAin    =  CAin*Q
W       =  FAin*X2*rholiq/r
print('Min Cat Weight= '+str(W)+' kg')

Min Cat Weight= 7.144037812469448e+18 kg


In [129]:
Cooling = T1 - Tin
print('Cooling required=' + str(Cooling)+'K')

Cooling required=88.22373890019526K


In [130]:
EnergyLoss = CpA*(Tin - T1)
EnergyLoss

-7057.899112015621

In [131]:
print('Inter-cooling required= '+str(EnergyLoss/1000)+' kW')

Inter-cooling required= -7.057899112015621 kW


---------------------------------------------------------

# <ins>Question 2 [16] </ins>

<img src="CRO410Supp2020b.JPG" width="750" />

The first order reaction where $A$ is converted to $B$ is carried out in a laboratory scale CSTR at different stirrer speeds (in rpm), at different temperatures and catalyst particle sizes. The results at a fixed concentration of $A$, $C_{A} = 16.9 x 10^{3}~mol/m3$, is shown on the attached figure. The catalyst is porous and $\rho_{cat} = 1200~kg/m^{3}$. It is also known that the total catalyst surface area ($S_{i}$) is $100~m^{2}/g$.

*a) Quantify the mass based intrinsic rate constant for this reaction (i.e. calculate the pre-exponential constant and the activation energy)* [4]

In [132]:
CAb    = 16.9e3 #mol/m3
rhocat =   1200 #kg/m3
R      =  8.314 #J/kg.K
Si     =    100 #m2/g

For $d_{p} = 0.1~mm$ and $d_{p} = 0.3~mm$, $-r_{A}^{'} = 0.1355~mol/(kg.s)$ where $rpm > 400$  (*read off the graph*) Different particle sizes and the same reaction rate implies negligible external mass transfer and $\eta = 1$, therefore, $C_{A_{b}} = C_{A_{s}}$

Rate equation:
\begin{align}
r_{A}^{'} &= k_{i}^{'}C_{A_{b}} \nonumber
\end{align}

In [133]:
kprm400 = 0.1355/CAb
kprm400   #m3/(kg.s)

8.017751479289941e-06

The intrinsic rate constant above is for $400~K$, which means it is also valid for $d_{p} = 5~mm$

1) We know at higher stirrer speeds, $k_{i}^{'}$ does not depend on particle diameter for the same temperature. And is NOT proportional to $d_{p}$ when porous catalysts are used and internal mass transfer dominates.

2) We also know, because both $d_{p} = 0.3~mm$ and $d_{p} = 0.1~mm$ converge at high stirrer speeds (i.e. both have the same $-r_{A}$), that $\eta = 1$ at $400~K$ for those two. Since $d_{p} = 5~mm$ has a lower $-r_{A}$ at higher stirrer speeds, it's $\eta$ must be less than $1$

3) Using the intrinsic rate equation, $-r_{A} = \eta k_{i}^{'}C_{A_{b}}$, we have two equations:
\begin{align}
0.1355 &= k_{1}^{'}C_{A_{b}} \nonumber \\
0.1000 &= \eta k_{2}^{'}C_{A_{b}} \nonumber \\
k_{1}^{'} &= k_{2}^{'} \nonumber
\end{align}

In [134]:
# therefore, eta for dp=5mm:
eta400 = 0.1/0.1355
eta400

0.7380073800738007

The pre-exponential mass-based rate constant, $k_{0}^{'}$ is the same for all $d_{p}$ in this reaction. Therefore, to calculate the activation energy, we need two mass-based intrinsic rate constants:
\begin{align}
k_{1}^{'} &= k_{0}^{'}e^{\dfrac{-E_{a}}{RT_{1}}} \nonumber \\
k_{2}^{'} &= k_{0}^{'}e^{\dfrac{-E_{a}}{RT_{2}}} \nonumber \\
ln\left(\dfrac{k_{1}^{'}}{k_{2}^{'}}\right) &= \dfrac{E_{a}}{R}\left[\dfrac{1}{T_{2}} - \dfrac{1}{T_{1}} \right] \nonumber \\
\therefore E_{a} &= R\left[\dfrac{1}{T_{2}} - \dfrac{1}{T_{1}} \right]^{-1}ln\left(\dfrac{k_{1}^{'}}{k_{2}^{'}}\right) \nonumber
\end{align}

In [135]:
kprm300 = 0.018/CAb #for dp=0.5mm and dp=1mm at high rpms
kprm300

1.0650887573964497e-06

In [136]:
Ea = R*log(kprm400/kprm300)/((1/300)-(1/400))
Ea #J/mol

20139.167306963678

In [137]:
koprm = kprm400*exp(Ea/(R*400))
koprm

0.003420211484307145

In [138]:
print('The pre-exponential rate constant= '+str(koprm*1000)+'e-3 m3/kg/s ')

The pre-exponential rate constant= 3.420211484307145e-3 m3/kg/s 


In [139]:
print('The activation energy= '+str(Ea/1000)+' kJ/mol')

The activation energy= 20.139167306963678 kJ/mol


*b) In addition to its dependence on particle diameter, it is known that the specific mass transfer coefficient, ($k_{c}$), in the CSTR where the studies were done, is related to the stirrer speed such that $k_{c} \propto (rpm)^{n}$. Determine the value of $n$* [6]

In [140]:
dp       = array([0.1/1000, 0.3/1000, 5.0/1000, 0.5/1000, 1.0])
rAprm100 = array([0.132, 0.119, 0.013, 0.0175, 0.0129])
rAprm200 = array([0.135, 0.133, 0.0545, 0.018, 0.0175])

\begin{align}
a_{m} &= \dfrac{6}{d_{p}}\dfrac{1}{\rho_{cat}} \nonumber 
\end{align}

In [141]:
am = 6/(dp*rhocat)
am

array([5.00000000e+01, 1.66666667e+01, 1.00000000e+00, 1.00000000e+01,
       5.00000000e-03])

For first order reactions:
\begin{align}
\dfrac{1}{k_{eff}^{'}} &= \dfrac{1}{k_{c}a_{m}} + \dfrac{1}{\eta k_{i}^{'}} \nonumber \\
\dfrac{1}{k_{c}a_{m}} &=\left[ \dfrac{1}{k_{eff}^{'}} - \dfrac{1}{\eta k_{i}^{'}} \right] \nonumber \\
k_{c} &= \dfrac{1}{a_{m}}\left[ \dfrac{1}{k_{eff}^{'}} - \dfrac{1}{\eta k_{i}^{'}} \right]^{-1} \nonumber 
\end{align}

In [142]:
kprmeff100 = rAprm100/CAb
kprmeff100

array([7.81065089e-06, 7.04142012e-06, 7.69230769e-07, 1.03550296e-06,
       7.63313609e-07])

In [143]:
kprmeff200 = rAprm200/CAb
kprmeff200

array([7.98816568e-06, 7.86982249e-06, 3.22485207e-06, 1.06508876e-06,
       1.03550296e-06])

Approach: will take $d_{p}=0.1~mm$ and $d_{p} = 0.3~mm$ at $rpm = 100$ and $rpm = 200$ respectively, to calculate $k_{c}$ at each point. Both these particle sizes have the same temperature, the same intrinsic rate constant and both have $\eta = 1$

Therefore:
\begin{align}
\dfrac{k_{c2}}{k_{c1}} &= \left( \dfrac{rpm_{2}}{rpm_{1}} \right)^{n} \nonumber \\
ln\left(\dfrac{k_{c2}}{k_{c1}}\right) &= n.ln\left( \dfrac{rpm_{2}}{rpm_{1}} \right) \nonumber \\
n &= \dfrac{ln\left(\dfrac{k_{c2}}{k_{c1}}\right)}{ln\left( \dfrac{rpm_{2}}{rpm_{1}} \right)} \nonumber
\end{align}

In [144]:
rpm1 = 100
rpm2 = 200

Using data for $d_{p}=0.1~mm$:

In [145]:
kc1 = (am[0]*((1/kprmeff100[0])-(1/kprm400)))**(-1)
kc2 = (am[0]*((1/kprmeff200[0])-(1/kprm400)))**(-1)

In [157]:
d1 = log(kc2/kc1)
e1 = log(rpm2/rpm1)
n1 = d1/e1
n1

2.839776399749976

Using data for $d_{p}=0.3~mm$ to check:

In [147]:
kc3 = (am[1]*((1/kprmeff100[1])-(1/kprm400)))**(-1)
kc4 = (am[1]*((1/kprmeff200[1])-(1/kprm400)))**(-1)

In [158]:
d2 = log(kc4/kc3)
e2 = log(rpm2/rpm1)
n2 = d2/e2
n2

2.8829306966643498

Using data for $d_{p}=5~mm$ to check:

In [159]:
kc5 = (am[2]*((1/kprmeff100[2])-(1/(eta400*kprm400))))**(-1)
kc6 = (am[2]*((1/kprmeff200[2])-(1/(eta400*kprm400))))**(-1)

In [160]:
d3 = log(kc6/kc5)
e3 = log(rpm2/rpm1)
n3 = d3/e3
n3

3.0028934622858663

In [149]:
print('Therefore, n = 2.85')

Therefore, n = 2.84


*c) Calculate the temperature above which it can be assumed that the apparent rate is in the internal mass transfer limited regime for the $5~mm$ catalyst particles. (Assume $D_{e}$ to be independent of temperature)* [6]

In [150]:
dp5 = dp[2]
R5  = dp5/2

In question a) we worked out the effectiveness factor where $\eta = 0.738$. In order to calculate $\phi$, we use:
\begin{align}
\eta = \dfrac{3}{\phi_{1}^{2}}(\phi_{1}coth\phi_{1} - 1) \nonumber
\end{align}

In [151]:
phi = 3/eta400 #1st guess
error = 0.1

In [152]:
while error >= 0.0000000001:
    phinew  = ((3/eta400)*((phi/tanh(phi)) - 1))**0.5
    error    = abs(phi - phinew)
    phi      = phinew

In [153]:
phi400 = phi
print(str(phi400))

2.48725078247475


We know that where $\phi_{1} > 20$, the reaction is in the internal diffusion limited regime. We use this knowledge to calculate the temperature. Both real and apparent rates will share that same temperature.

From the Thiele equation:
\begin{align}
\dfrac{\phi_{1}^{2})_{1}}{\phi_{1}^{2})_{2}} &= \dfrac{k_{1}^{'})}{k_{2}^{'})} \nonumber
\end{align}

In [154]:
phi20  = 20
kprm20 = kprm400*(phi20/phi400)**2
kprm20

0.0005184100706903991

From the Arrhenius equation:
\begin{align}
k_{i}^{'} &= k_{0}^{'}e^{\left(\dfrac{-E_{a}}{RT} \right)} \nonumber \\
ln\left(\dfrac{k_{0}^{'}}{k_{i}^{'}}\right) &= \left(\dfrac{E_{a}}{RT} \right) \nonumber \\
T &= ln\left(\dfrac{k_{0}^{'}}{k_{i}^{'}}\right)^{-1}\left(\dfrac{E_{a}}{R} \right) \nonumber
\end{align}

In [155]:
y = 1/log(koprm/kprm20)
z = Ea/R
T = y*z
T

1283.8984962578809

In [156]:
print('IMLR is assumed above '+str(T)+' K')

IMLR is assumed above 1283.8984962578809 K
